In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

In [2]:
# Reading files

# Read in the data
# 2018
ela_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM ELA.csv"
)
math_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM Math.csv"
)
science_2018 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2018\Annual EM SCIENCE.csv"
)



C:\Users\ericr\AppData\Local\Temp\ipykernel_36620\1614234590.py:5: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ela_2018 = pd.read_csv(
C:\Users\ericr\AppData\Local\Temp\ipykernel_36620\1614234590.py:8: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2018 = pd.read_csv(
C:\Users\ericr\AppData\Local\Temp\ipykernel_36620\1614234590.py:11: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  science_2018 = pd.read_csv(


In [3]:
# Checking types
ela_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654694 entries, 0 to 654693
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ENTITY_CD           654694 non-null  int64 
 1   ENTITY_NAME         654694 non-null  object
 2   YEAR                654694 non-null  int64 
 3   ASSESSMENT_NAME     654694 non-null  object
 4   SUBGROUP_NAME       654694 non-null  object
 5   NUM_TESTED          654694 non-null  int64 
 6   NOT_TESTED          654694 non-null  int64 
 7   LEVEL1_COUNT        654694 non-null  object
 8   LEVEL1_%TESTED      654694 non-null  object
 9   LEVEL2_COUNT        654694 non-null  object
 10  LEVEL2_%TESTED      654694 non-null  object
 11  LEVEL3_COUNT        654694 non-null  object
 12  LEVEL3_%TESTED      654694 non-null  object
 13  LEVEL4_COUNT        654694 non-null  object
 14  LEVEL4_%TESTED      654694 non-null  object
 15  LEVEL5_COUNT        304561 non-null  object
 16  LE

In [4]:
# Our columns of interest
Team_9_keep_cols = ["NUM_TESTED", "NUM_PROF", "TOTAL_SCALE_SCORES", "MEAN_SCORE"]

In [5]:
# Scouting racial groups
ela_2018.SUBGROUP_NAME.unique()

array(['Hispanic or Latino', 'Homeless', 'Male',
       'Non-English Language Learners', 'Not Economically Disadvantaged',
       'American Indian or Alaska Native',
       'Asian or Native Hawaiian/Other Pacific Islander',
       'Black or African American', 'Economically Disadvantaged',
       'English Language Learners', 'Female', 'General Education',
       'Multiracial', 'All Students', 'Not Migrant',
       'Students with Disabilities', 'White', 'Migrant',
       'Small Group Total', 'American Indian or Alaska Native Female',
       'American Indian or Alaska Native Male',
       'Asian or Native Hawaiian/Other Pacific Islander Female',
       'Asian or Native Hawaiian/Other Pacific Islander Male',
       'Black or African American Female',
       'Black or African American Male', 'Hispanic or Latino Female',
       'Hispanic or Latino Male', 'Multiracial Female',
       'Multiracial Male', 'White Female', 'White Male', 'Not Homeless',
       'Not in Foster Care', 'Parent in Arme

In [6]:


def sanity_check_entity_cd(ds):
    
    for i in ds.ENTITY_CD.unique():
        try: 
            int(i)
        except:
            print(i)

In [7]:
def race_df_maker(df : pd.DataFrame, base_cols : list[str], subgroups : list[str], keep_cols : list[str] = []) -> pd.DataFrame:
    """Takes a our base dataframe and stacks the subgroups horizontally
    via a merge.
    """

    copy_df = df.copy() # to contain all the core records

    copy_df['Subject'] = copy_df['ASSESSMENT_NAME'].str.extract(r"([A-Za-z]*).*", expand=False)

    copy_df[copy_df.MEAN_SCORE == 's'] = np.nan

    copy_df['MEAN_SCORE'] = copy_df['MEAN_SCORE'].astype(float)

    copy_df["MEAN_SCORE"].fillna(copy_df.groupby(["YEAR", "ENTITY_CD", "Subject"])["MEAN_SCORE"].transform("mean"), inplace=True)

    base_cols.append("Subject")
    
    if keep_cols != []:
        new_cols = keep_cols + base_cols + ["SUBGROUP_NAME"]

    else:

        new_cols = base_cols + ["SUBGROUP_NAME"]

    base_df = copy_df[new_cols].copy() # get the core records

    base_df.drop_duplicates(subset = base_cols, inplace=True, ignore_index=True) # remove duplicates

    core_df = copy_df[new_cols].copy() # get the core records

    # At this point, we have a df custom to merge with Team 9's base data
    # that contains all the schools,
    # But we still need the data by race

    # We will create a dictionary of dataframes

    for group in subgroups:

        # copy to avoid pointer issues
        
        group_df = core_df[core_df["SUBGROUP_NAME"] == group].copy()

        formatted_name = group.replace(" ", "_").lower()

        base_df = base_df.merge(group_df, on=base_cols, how="left", suffixes=("", f"_{formatted_name}"))

    # Removing the subgroup name columns

    ls_subgroup_cols = [col for col in base_df.columns if "subgroup_name" in col.lower()]

    extra_cols = ['NUM_TESTED', 'NUM_PROF', 'TOTAL_SCALE_SCORES', 'MEAN_SCORE']

    base_df.drop(columns=ls_subgroup_cols + extra_cols, inplace=True)
    
    return base_df

In [8]:
# testing the function
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    [
        "Hispanic or Latino",
        "Black or African American",
        "American Indian or Alaska Native",
        "Asian or Native Hawaiian/Other Pacific Islander",
        "Multiracial",
        "White",
    ], keep_cols=Team_9_keep_cols
).keys()

Index(['ENTITY_CD', 'YEAR', 'ASSESSMENT_NAME', 'Subject',
       'NUM_TESTED_hispanic_or_latino', 'NUM_PROF_hispanic_or_latino',
       'TOTAL_SCALE_SCORES_hispanic_or_latino',
       'MEAN_SCORE_hispanic_or_latino', 'NUM_TESTED_black_or_african_american',
       'NUM_PROF_black_or_african_american',
       'TOTAL_SCALE_SCORES_black_or_african_american',
       'MEAN_SCORE_black_or_african_american',
       'NUM_TESTED_american_indian_or_alaska_native',
       'NUM_PROF_american_indian_or_alaska_native',
       'TOTAL_SCALE_SCORES_american_indian_or_alaska_native',
       'MEAN_SCORE_american_indian_or_alaska_native',
       'NUM_TESTED_asian_or_native_hawaiian/other_pacific_islander',
       'NUM_PROF_asian_or_native_hawaiian/other_pacific_islander',
       'TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander',
       'MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander',
       'NUM_TESTED_multiracial', 'NUM_PROF_multiracial',
       'TOTAL_SCALE_SCORES_multirac

In [9]:
race_df_maker(
    ela_2018,
    ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
    [
        "Hispanic or Latino",
        "Black or African American",
        "American Indian or Alaska Native",
        "Asian or Native Hawaiian/Other Pacific Islander",
        "Multiracial",
        "White",
    ], keep_cols=Team_9_keep_cols
).shape

(36740, 28)

### 2018

In [10]:
# testing the function
for ds in [ela_2018, math_2018, science_2018]:
    sanity_check_entity_cd(ds)
    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            [
                "Hispanic or Latino",
                "Black or African American",
                "American Indian or Alaska Native",
                "Asian or Native Hawaiian/Other Pacific Islander",
                "Multiracial",
                "White",
            ], keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)

In [11]:
final_ds = None

for ds in [ela_2018, math_2018, science_2018]:

    assert 2018 in ds.YEAR.unique()
    
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        [
            "Hispanic or Latino",
            "Black or African American",
            "American Indian or Alaska Native",
            "Asian or Native Hawaiian/Other Pacific Islander",
            "Multiracial",
            "White",
        ],
        keep_cols=Team_9_keep_cols,
    )

    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

In [12]:

# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")

Shape of final_ds: (96178, 28)



In [13]:
final_ds.head(10)

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_hispanic_or_latino,NUM_PROF_hispanic_or_latino,TOTAL_SCALE_SCORES_hispanic_or_latino,MEAN_SCORE_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_PROF_black_or_african_american,...,TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,NUM_TESTED_multiracial,NUM_PROF_multiracial,TOTAL_SCALE_SCORES_multiracial,MEAN_SCORE_multiracial,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white
0,3.314000e+11,2018.0,ELA8,ELA,39.0,7,22910,587.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.314000e+11,2018.0,ELA7,ELA,36.0,6,21346,593.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.314000e+11,2018.0,ELA6,ELA,36.0,15,21466,596.0,12.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.314000e+11,2018.0,ELA8,ELA,24.0,10,14345,598.0,14.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.314000e+11,2018.0,ELA7,ELA,NaN,NaN,NaN,NaN,24.0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3.314000e+11,2018.0,ELA6,ELA,23.0,6,13595,591.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.000000e+00,2017.0,ELA3,ELA,28638.0,NaN,8572251,299.0,14688.0,NaN,...,3968338,323.0,847.0,NaN,279413,330.0,11234.0,NaN,3642798,324.0
8,1.000000e+00,2018.0,ELA5,ELA,28234.0,7670,16817855,596.0,14811.0,3749,...,7783323,610.0,807.0,509,494431,613.0,11063.0,6383,6745329,610.0
9,1.000000e+00,2018.0,ELA3_8,ELA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# delete the dataframes to save memory
del ela_2018, math_2018, science_2018
gc.collect()

21

### 2019

In [15]:
# 2019

ela_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM ELA.csv"
)
math_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM MATH.csv"
)
science_2019 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2019\Annual EM SCIENCE.csv"
)

# testing the function
for ds in [ela_2019, math_2019, science_2019]:
    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            [
                "Hispanic or Latino",
                "Black or African American",
                "American Indian or Alaska Native",
                "Asian or Native Hawaiian/Other Pacific Islander",
                "Multiracial",
                "White",
            ], keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)



In [16]:
for ds in [ela_2019, math_2019, science_2019]:

    assert 2019 in ds.YEAR.unique()
    
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        [
            "Hispanic or Latino",
            "Black or African American",
            "American Indian or Alaska Native",
            "Asian or Native Hawaiian/Other Pacific Islander",
            "Multiracial",
            "White",
        ],
        keep_cols=Team_9_keep_cols,
    )

    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

In [17]:

# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")

Shape of final_ds: (210576, 28)



In [18]:
final_ds.tail(10)

,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_hispanic_or_latino,NUM_PROF_hispanic_or_latino,TOTAL_SCALE_SCORES_hispanic_or_latino,MEAN_SCORE_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_PROF_black_or_african_american,...,TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,NUM_TESTED_multiracial,NUM_PROF_multiracial,TOTAL_SCALE_SCORES_multiracial,MEAN_SCORE_multiracial,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white
210566,6.806011e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210567,6.806011e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210568,6.806011e+11,2019.0,Science4,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210569,6.808010e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210570,6.808010e+11,2019.0,Science4,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.0,50,4621,78.0
210571,6.808010e+11,2019.0,Science8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210572,6.808010e+11,2019.0,Science8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210573,6.808010e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210574,6.808010e+11,2019.0,Science4,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.0,50,4621,78.0
210575,6.808010e+11,2019.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# delete the dataframes to save memory
del ela_2019, math_2019, science_2019
gc.collect()

21

### 2020 does not exist

### 2021

In [20]:



# 2020 is unobserved

# 2021
ela_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM ELA.csv"
)
math_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM Math.csv"
)
science_2021 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2021\Annual EM SCIENCE.csv"
)


# testing the function

for ds in [ela_2021, math_2021, science_2021]:
    
    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            [
                "Hispanic or Latino",
                "Black or African American",
                "American Indian or Alaska Native",
                "Asian or Native Hawaiian/Other Pacific Islander",
                "Multiracial",
                "White",
            ], keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(ds.YEAR.unique(), ds.ASSESSMENT_NAME.unique())
        print(e)
        print(ds.head())
        print(ds.shape)
        continue

C:\Users\ericr\AppData\Local\Temp\ipykernel_36620\1940774578.py:7: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2021 = pd.read_csv(


[2021] ['ELA3' 'ELA4' 'ELA5' 'ELA6' 'ELA7' 'ELA8' 'ELA3_8']
'DataFrame' object has no attribute 'MEAN_SCORE'
   INSTITUTION_ID     ENTITY_CD         ENTITY_NAME  YEAR ASSESSMENT_NAME   
0    8.000001e+11  111111111111  All Public Schools  2021            ELA3  \
1    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
2    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
3    8.000001e+11  111111111111  All Public Schools  2021            ELA3   
4    8.000001e+11  111111111111  All Public Schools  2021            ELA3   

                      SUBGROUP_NAME  TOTAL_COUNT  NOT_TESTED  PCT_NOT_TESTED   
0                      All Students       192560       99642              52  \
1                            Female        93485       47472              51   
2                              Male        99075       52170              53   
3  American Indian or Alaska Native         1469         963              66   
4         Black or African A

In [21]:
math_2021.keys()

Index(['INSTITUTION_ID', 'ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'ASSESSMENT_NAME',
       'SUBGROUP_NAME', 'TOTAL_COUNT', 'NOT_TESTED', 'PCT_NOT_TESTED',
       'NUM_TESTED', 'PCT_TESTED', 'LEVEL1_COUNT', 'LEVEL1_%TESTED',
       'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT', 'LEVEL3_%TESTED',
       'LEVEL4_COUNT', 'LEVEL4_%TESTED', 'LEVEL5_COUNT', 'LEVEL5_%TESTED',
       'NUM_PROF', 'PER_PROF', 'TOTAL_EXEMPT', 'NUM_EXEMPT_NTEST',
       'PCT_EXEMPT_NTEST', 'NUM_EXEMPT_TEST', 'PCT_EXEMPT_TEST'],
      dtype='object')

>2021 has no mean scores. The message propagates across all 3 subjects. 

In [22]:
# delete the dataframes to save memory
del ela_2021, math_2021, science_2021
gc.collect()

57

### 2022

In [23]:


# 2022

ela_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM ELA.csv"
)
math_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM Math.csv"
)
science_2022 = pd.read_csv(
    r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\Data-20230420T224029Z-001\Data\2022\Annual EM SCIENCE.csv"
)

# testing the function
for ds in [ela_2022, math_2022, science_2022]:

    sanity_check_entity_cd(ds)

    try: 
        race_df_maker(
            ds,
            ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"], # ENTITY_NAME
            [
                "Hispanic or Latino",
                "Black or African American",
                "American Indian or Alaska Native",
                "Asian or Native Hawaiian/Other Pacific Islander",
                "Multiracial",
                "White",
            ], keep_cols=Team_9_keep_cols
        )

    except Exception as e:
        print(e)
        print(ds.head())
        print(ds.shape)

C:\Users\ericr\AppData\Local\Temp\ipykernel_36620\591796911.py:6: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  math_2022 = pd.read_csv(


In [24]:
for ds in [ela_2022, math_2022, science_2022]:

    assert 2022 in ds.YEAR.unique()
    
    new_ds = race_df_maker(
        ds,
        ["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],  # ENTITY_NAME
        [
            "Hispanic or Latino",
            "Black or African American",
            "American Indian or Alaska Native",
            "Asian or Native Hawaiian/Other Pacific Islander",
            "Multiracial",
            "White",
        ],
        keep_cols=Team_9_keep_cols,
    )

    if final_ds is None:

        final_ds = new_ds.copy()

    else:

        final_ds = pd.concat([final_ds, new_ds], ignore_index=True)

In [25]:
# sanity checks

print("Shape of final_ds:", final_ds.shape)
print("")
final_ds.tail(10)

Shape of final_ds: (339913, 28)



,ENTITY_CD,YEAR,ASSESSMENT_NAME,Subject,NUM_TESTED_hispanic_or_latino,NUM_PROF_hispanic_or_latino,TOTAL_SCALE_SCORES_hispanic_or_latino,MEAN_SCORE_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_PROF_black_or_african_american,...,TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,NUM_TESTED_multiracial,NUM_PROF_multiracial,TOTAL_SCALE_SCORES_multiracial,MEAN_SCORE_multiracial,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white
339903,6.806011e+11,2022.0,Science4,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,70.0,63,5795,83.0
339904,6.808010e+11,2022.0,Science8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339905,6.808010e+11,2022.0,CombinedScience,CombinedScience,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339906,6.808010e+11,2022.0,Science4,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,43.0,39,3371,78.0
339907,6.808010e+11,2022.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339908,6.808010e+11,2022.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339909,6.808010e+11,2022.0,CombinedScience,CombinedScience,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339910,6.808010e+11,2022.0,Science8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339911,6.808010e+11,2022.0,Science4_8,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339912,6.808010e+11,2022.0,Science4,Science,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,42.0,38,3306,79.0


In [26]:
# delete the dataframes to save memory
del ela_2022, math_2022, science_2022

### Merging with Team 9's All Data CSV

In [27]:
all_data = pd.read_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\final.csv")

In [28]:
# sanity checks
print("Shape of all_data:", all_data.shape)

Shape of all_data: (2933498, 4)


In [29]:

all_data.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME
0,1,2018,ELA,ELA5
1,1,2018,ELA,ELA3_8
2,1,2018,ELA,ELA3
3,1,2018,ELA,ELA7
4,1,2018,ELA,ELA8


In [30]:
full_data = pd.merge(all_data, final_ds, how="left", on=["ENTITY_CD", "YEAR", "Subject", "ASSESSMENT_NAME"])
full_data.head()

,ENTITY_CD,YEAR,Subject,ASSESSMENT_NAME,NUM_TESTED_hispanic_or_latino,NUM_PROF_hispanic_or_latino,TOTAL_SCALE_SCORES_hispanic_or_latino,MEAN_SCORE_hispanic_or_latino,NUM_TESTED_black_or_african_american,NUM_PROF_black_or_african_american,...,TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander,MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander,NUM_TESTED_multiracial,NUM_PROF_multiracial,TOTAL_SCALE_SCORES_multiracial,MEAN_SCORE_multiracial,NUM_TESTED_white,NUM_PROF_white,TOTAL_SCALE_SCORES_white,MEAN_SCORE_white
0,1,2018,ELA,ELA5,28234.0,7670,16817855,596.0,14811.0,3749,...,7783323,610.0,807.0,509,494431,613.0,11063.0,6383,6745329,610.0
1,1,2018,ELA,ELA5,28234.0,7670,16817855,596.0,14811.0,3749,...,7783323,610.0,807.0,509,494431,613.0,11063.0,6383,6745329,610.0
2,1,2018,ELA,ELA3_8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2018,ELA,ELA3_8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2018,ELA,ELA3,28172.0,11285,16769386,595.0,13861.0,5374,...,7375296,610.0,937.0,675,572429,611.0,11416.0,7869,6950213,609.0


In [31]:
full_data.keys()

Index(['ENTITY_CD', 'YEAR', 'Subject', 'ASSESSMENT_NAME',
       'NUM_TESTED_hispanic_or_latino', 'NUM_PROF_hispanic_or_latino',
       'TOTAL_SCALE_SCORES_hispanic_or_latino',
       'MEAN_SCORE_hispanic_or_latino', 'NUM_TESTED_black_or_african_american',
       'NUM_PROF_black_or_african_american',
       'TOTAL_SCALE_SCORES_black_or_african_american',
       'MEAN_SCORE_black_or_african_american',
       'NUM_TESTED_american_indian_or_alaska_native',
       'NUM_PROF_american_indian_or_alaska_native',
       'TOTAL_SCALE_SCORES_american_indian_or_alaska_native',
       'MEAN_SCORE_american_indian_or_alaska_native',
       'NUM_TESTED_asian_or_native_hawaiian/other_pacific_islander',
       'NUM_PROF_asian_or_native_hawaiian/other_pacific_islander',
       'TOTAL_SCALE_SCORES_asian_or_native_hawaiian/other_pacific_islander',
       'MEAN_SCORE_asian_or_native_hawaiian/other_pacific_islander',
       'NUM_TESTED_multiracial', 'NUM_PROF_multiracial',
       'TOTAL_SCALE_SCORES_multirac

In [ ]:
full_data.to_csv(r"C:\Users\ericr\Desktop\IDS 701 - Unifying Data Science\uds final project\unifying-data-science-2023-project-grupo9\full_data_race.csv", index=False)

In [ ]:
full_data.isnull().sum()

na_cols = full_data.isnull().sum()[full_data.isnull().sum() > 0].index

In [ ]:
full_data.isnull().sum()


In [ ]:
test_copy = full_data.copy()

In [ ]:

for col in na_cols:

    test_copy[col].fillna(test_copy.groupby(["YEAR","ENTITY_CD","Subject", "ASSESSMENT_NAME"])[col].transform("median"), inplace=True)

In [ ]:
test_copy.head()

In [ ]:
test_copy.isnull().sum()

In [ ]:
full_data

full_data["MEAN_SCORE"].fillna(copy_df.groupby(["YEAR", "ENTITY_CD", "Subject"])["MEAN_SCORE"].transform("mean"), inplace=True)
